In [3]:
!pip install -U langgraph langchain langchain_openai langchain_experimental langsmith pandas

ERROR: Exception:
Traceback (most recent call last):
  File "C:\Users\anish\AppData\Local\Programs\Python\Python311\Lib\site-packages\pip\_internal\cli\base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "C:\Users\anish\AppData\Local\Programs\Python\Python311\Lib\site-packages\pip\_internal\cli\req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\anish\AppData\Local\Programs\Python\Python311\Lib\site-packages\pip\_internal\commands\install.py", line 377, in run
    requirement_set = resolver.resolve(
                      ^^^^^^^^^^^^^^^^^
  File "C:\Users\anish\AppData\Local\Programs\Python\Python311\Lib\site-packages\pip\_internal\resolution\resolvelib\resolver.py", line 136, in resolve
    elif installed_dist.version != candidate.version:
         ^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\anish\AppData\Local\Programs\Python\Python311\Lib\site-packa


  Using cached langgraph-0.2.44-py3-none-any.whl.metadata (15 kB)
  Using cached langchain-0.3.7-py3-none-any.whl.metadata (7.1 kB)
  Using cached langchain_openai-0.2.5-py3-none-any.whl.metadata (2.6 kB)
  Using cached langchain_experimental-0.3.3-py3-none-any.whl.metadata (1.7 kB)
  Using cached langsmith-0.1.139-py3-none-any.whl.metadata (13 kB)
  Using cached langchain_core-0.3.15-py3-none-any.whl.metadata (6.3 kB)
  Using cached langgraph_checkpoint-2.0.2-py3-none-any.whl.metadata (4.6 kB)
  Using cached langgraph_sdk-0.1.35-py3-none-any.whl.metadata (1.8 kB)
  Using cached langchain_text_splitters-0.3.2-py3-none-any.whl.metadata (2.3 kB)
  Using cached openai-1.53.0-py3-none-any.whl.metadata (24 kB)
  Using cached langchain_community-0.3.5-py3-none-any.whl.metadata (2.9 kB)
  Using cached pydantic_settings-2.6.1-py3-none-any.whl.metadata (3.5 kB)


In [5]:
!pip show langgraph

Name: langgraph
Version: 0.2.44
Summary: Building stateful, multi-actor applications with LLMs
Home-page: https://www.github.com/langchain-ai/langgraph
Author: 
Author-email: 
License: MIT
Location: C:\Users\anish\AppData\Local\Programs\Python\Python311\Lib\site-packages
Requires: langchain-core, langgraph-checkpoint, langgraph-sdk
Required-by: 


In [36]:
import getpass
import os

def set_api_key(var: str):
    os.environ[var] = getpass.getpass(f"Please provide your {var}")

# Prompt for API keys every time
set_api_key("OPENAI_API_KEY")
set_api_key("TAVILY_API_KEY")
set_api_key("LANGCHAIN_API_KEY")


In [ ]:
import os

# Set environment variables
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"
os.environ["LANGCHAIN_PROJECT"] = "Multi Agent"


cREATE TOOLS - ONE TOOL FOR SEARCH ENGINE IN A WEB SEARCH , ONE TO CREATE PLOTS

In [54]:
from typing import Annotated

from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_experimental.tools import PythonREPLTool

tavily_tool = TavilySearchResults(max_results = 5)
python_repl_tool = PythonREPLTool()

### Helper Function
To convert the agent response to human message which is  to add to the global state of the graph

In [55]:
from langchain_core.messages import HumanMessage

def agent_node(state,agent,name):
    result = agent.invoke(state)
    return {
        "messages": [HumanMessage(content = result["messages"][-1].content,name = name)]
    }

### Creating the Agent Supervisor

In [56]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
from pydantic import BaseModel
from typing import Literal

members = ["Researcher","Coder"]
system_prompt = (
    "You are a supervisor tasked with managing a conversation between the"
    "following workers : {members}. Given the following user request,"
    "respond with the worker to act next. Each worker will perform a"
    "task and respond with their results and status. When finished,"
    "respond with FINISH."
)

#Our team supervisor is an LLM node. It just picks the next agent to process and decides when the work is completed
options = ["FINISH"] + members

class routeResponse(BaseModel):
    next: Literal[*options]

prompt  = ChatPromptTemplate(
    [
        ("system",system_prompt),
        MessagesPlaceholder(variable_name = "messages"),
        (
            "system",
            "Given the conversation above, who should act next?"
            "Or should we FINISH? Select one of {options}",
        ),
    ],
    ).partial(options = str(options),members = ", ".join(members))

llm =ChatOpenAI(model = "gpt-4o")

def supervisor_Agent(state):
    supervisor_chain = prompt|llm.with_structured_output(routeResponse)
    return supervisor_chain.invoke(state)

### Constructing Graph
Defining the worker nodes using the function we defined

In [57]:
import functools 
import operator
from typing import Sequence
from typing_extensions import TypedDict

from langchain_core.messages import BaseMessage

from langgraph.graph import END, StateGraph, START
from langgraph.prebuilt import create_react_agent

class AgentState(TypedDict):
# The annotation tells the graph that new messages will always be added to the current states
    messages: Annotated[Sequence[BaseMessage],operator.add]
    # The next field indicates where to route next
    next: str

research_agent = create_react_agent(llm, tools = [tavily_tool])
research_node = functools.partial(agent_node, agent = research_agent, name = "Researcher")

code_agent = create_react_agent(llm, tools = [python_repl_tool])
code_node = functools.partial(agent_node, agent = code_agent, name = "Coder")

workflow = StateGraph(AgentState)
workflow.add_node("Researcher",research_node) 
workflow.add_node("Coder",code_node)
workflow.add_node("supervisor",supervisor_Agent)                                             

Connecting all the Edges in a graph

In [58]:
for member in members:
    #Since we always want the agents workers to report back to the supervisor when done
    workflow.add_edge(member,"supervisor")
#The supervisor will then decide who to route to next or to finish it
conditional_map = {k: k for k in members}
conditional_map["FINISH"] = END
if "supervisor" not in workflow.branches:
    workflow.add_conditional_edges("supervisor", lambda x: x["next"], conditional_map)
workflow.add_edge(START,"supervisor")

graph = workflow.compile()

In [59]:
### Invoke the team
for s in graph.stream(
    {
        "messages"
: [
    HumanMessage(content = "code hello world and print it to the terminal")
]   }
):
    if "__end__" not in s:
        print(s)
        print("----")

{'supervisor': {'next': 'Coder'}}
----
{'Coder': {'messages': [HumanMessage(content='Here\'s the code to print "Hello, World!" to the terminal:\n\n```python\nprint(\'Hello, World!\')\n```\n\nWhen executed, it produces the output:\n\n```\nHello, World!\n```', additional_kwargs={}, response_metadata={}, name='Coder')]}}
----
{'supervisor': {'next': 'FINISH'}}
----


In [47]:
for s in graph.stream(
    {"messages": [HumanMessage(content = "Write a brief research report on Claude models")]},
    {"recursion_limit": 10},
):
    if "__end__" not in s:
        print(s)
        print("----")

{'supervisor': {'next': 'Researcher'}}
----
{'Researcher': {'messages': [HumanMessage(content="### Research Report on Claude Models\n\n#### Overview\nClaude models are a family of large language models (LLMs) developed by Anthropic, an AI research and safety company based in San Francisco. These models are designed to excel in general, open-ended conversation, search, writing, editing, outlining, and various forms of text classification applications.\n\n#### Claude 2\nClaude 2 was introduced earlier in 2023 and is noted for its capabilities in automating a wide variety of text classification tasks. It serves as a robust pre-trained model that offers a strong foundation for developing further AI solutions. The training data for Claude 2 is current up to early 2023, with about 10% dedicated to research-specific content. This model has been evaluated for safety, alignment, and capabilities, ensuring it meets high standards of performance and ethical use.\n\n#### Claude 3 Model Family\nThe